# Яндекс.Музыка
___________

**Общая информация:**

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки необходимо сравнить поведение пользователей двух столиц.

**Цель исследования:**

Проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования:**

Данные о поведении пользователей находятся в файле `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез необходимо произвести обзор данных. 
Далее - проверить данные на ошибки и оценить их влияние на исследование. Затем на этапе предобработки исправить самые критичные ошибки данных.
 
*Таким образом, исследование пройдёт в три этапа:*
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Импортируем необходимую для работы библиотеку `pandas`.

In [80]:
import pandas as pd

Прочитаем файл `yandex_music_project.csv` из папки `/datasets` и сохраним его в переменной `df`:

In [81]:
df = pd.read_csv('/datasets/yandex_music_project.csv')

Выведем на экран первые десять строк таблицы и общую информацию о ней.

In [82]:
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


* Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.


* Согласно документации к данным:
   * `userID` — идентификатор пользователя;
   * `Track` — название трека;  
   * `artist` — имя исполнителя;
   * `genre` — название жанра;
   * `City` — город пользователя;
   * `time` — время начала прослушивания;
   * `Day` — день недели.


* Названия колонок записаны с нарушением стиля:
   1. Строчные буквы сочетаются с прописными.
   2. Встречаются пробелы.
   3. Неиспользование "змеиного регистра" в наименовании колонок таблицы.


* Количество значений в столбцах различается, то есть в данных имеются пропущенные значения.


**Вывод:**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

Исправим стиль в заголовках столбцов, исключим пропуски. Затем проверим данные на дубликаты.

### Стиль заголовков

Выведем на экран названия столбцов:

In [84]:
print(list(df.columns))

['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day']


Исправим стилистические ошибки в названиях столбцов:
* несколько слов в названии запишем в «змеином_регистре»,
* все символы сделаем строчными,
* устраним пробелы.

In [85]:
df = df.rename(columns={'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})
print(list(df.columns))

['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day']


### Пропуски значений

Определим количество пропусков в процентном соотношении.

In [86]:
print(df.isna().mean()*100)

user_id     0.000000
track       1.891547
artist     11.068086
genre       1.840840
city        0.000000
time        0.000000
day         0.000000
dtype: float64


Пропуски имеются в столбцах `track`, `artist`, `genre`.

В `track` и `artist` пропуски не важны для работы. Поэтому заменим их явными обозначениями.

Пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Заполним и эти пропуски явными обозначениями и оценим, насколько они повредят расчётам. 

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`.

In [87]:
columns_to_replace = ['track', 'artist', 'genre'] 

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')
    
print(df.isna().mean())    

user_id    0.0
track      0.0
artist     0.0
genre      0.0
city       0.0
time       0.0
day        0.0
dtype: float64


### Дубликаты

Посчитаем явные дубликаты в таблице одной командой:

In [88]:
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 3826


 Удалим явные дубликаты с удалением старых индексов и формированием новых:

In [89]:
df = df.drop_duplicates().reset_index(drop=True)

print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 0


Теперь избавимся от неявных дубликатов в колонке `genre`. Название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведем на экран список уникальных названий жанров, отсортированный в алфавитном порядке. 

In [90]:
print(sorted(df['genre'].unique()))

['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans', 'alternative', 'alternativepunk', 'ambient', 'americana', 'animated', 'anime', 'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook', 'author', 'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill', 'chinese', 'choral', 'christian', 'christmas', 'classical', 'classicmetal', 'club', 'colombian', 'comedy', 'conjazz', 'contemporary', 'country', 'cuban', 'dance', 'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic', 'electropop', 'emo', 'entehno', 'epicmetal', 'estrada', 'ethnic', 'eurofolk', 'european', 'experimental', 'extrememetal', 'fado', 'fairyt

Выше встречаются следующие неявные дубликаты жанра хип-хоп:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишем функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

In [91]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

Вызовем `replace_wrong_genres()` и передадим ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [92]:
wrong_genres = ('hip','hop','hip-hop')
correct_genre = 'hiphop'

replace_wrong_genres(wrong_genres, correct_genre)

print('В жанрах встречается только:', 
      df[df['genre'].isin(['hip'
                           ,'hop'
                           , 'hip-hop'
                           , 'hiphop'])]['genre'].sort_values().unique())

В жанрах встречается только: ['hiphop']


**Вывод:**

Предобработка обнаружила три проблемы в данных:
- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Для упрощения работы с таблицей были исправлены заголовки, устранены дубликаты, пропущенные значения заменены на `'unknown'`. 

Далее перейдем к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. 

Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга;
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

Оцените активность пользователей в каждом городе. Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [93]:
print(df.groupby('city')['user_id'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу. 

In [94]:
print(df.groupby('day')['user_id'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию, которая объединит два эти расчёта.

In [95]:
def number_tracks (day,city): 
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [96]:
cities = ['Moscow', 'Saint-Petersburg']
days = ['Monday', 'Wednesday', 'Friday']

for city in cities:
    for day in days: 
        print(f'Количество прослушиваний в городе {city} в {day}: {number_tracks(day, city)}')

Количество прослушиваний в городе Moscow в Monday: 15740
Количество прослушиваний в городе Moscow в Wednesday: 11056
Количество прослушиваний в городе Moscow в Friday: 15945
Количество прослушиваний в городе Saint-Petersburg в Monday: 5614
Количество прослушиваний в городе Saint-Petersburg в Wednesday: 7003
Количество прослушиваний в городе Saint-Petersburg в Friday: 5895


Создадим таблицу с результатами.

In [97]:
columns = ['city', 'monday', 'wednesday', 'friday']
values = [
    ['Moscow', number_tracks('Monday','Moscow'), number_tracks('Wednesday','Moscow'), number_tracks('Friday','Moscow')],
    ['Saint-Petersburg', number_tracks('Monday','Saint-Petersburg'), number_tracks('Wednesday','Saint-Petersburg'), number_tracks('Friday','Saint-Petersburg')]
]
result = pd.DataFrame(data = values, columns = columns)
display(result)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Вывод:**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраним таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [98]:
moscow_general = df[df['city'] == 'Moscow']
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [99]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]      
    genre_df = genre_df[genre_df['time'] > time1]   
    genre_df = genre_df[genre_df['time'] < time2]  

    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending = False)
   
    return (genre_df_sorted.head(10))

Cравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [100]:
generals = [moscow_general, spb_general]

for general in generals:
    for city in cities:
        print(f"Распределение треков по жанрам в городе {city} в понедельник утром:\n {genre_weekday(general, 'Monday', '07:00', '11:00')}\n")
        print(f"Распределение треков по жанрам в городе {city} в пятницу вечером:\n {genre_weekday(general, 'Friday', '17:00', '23:00')}\n")

Распределение треков по жанрам в городе Moscow в понедельник утром:
 genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

Распределение треков по жанрам в городе Moscow в пятницу вечером:
 genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

Распределение треков по жанрам в городе Saint-Petersburg в понедельник утром:
 genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

Распределение треков по жанрам в городе Saint-Petersburg в пятницу вечером:
 genre
pop            713
rock          

**Вывод:**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования. Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [101]:
moscow_genres = (moscow_general.groupby('genre')['genre'].count()).sort_values(ascending = False)
display(moscow_genres.head(10))

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим то же и для Петербурга.

In [102]:
spb_genres = (spb_general.groupby('genre')['genre'].count()).sort_values(ascending = False)
display(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Вывод:**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Было установлено следующее:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 
   
   Первая гипотеза полностью подтвердилась.


2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

  Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.


3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

   Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.